In [1]:
%pip install -U litellm langchain_ollama langchain_community langchain_anthropic langchain-tavily langchain_experimental matplotlib langgraph

Note: you may need to restart the kernel to use updated packages.


In [2]:
!pip install langchain_openai

In [3]:
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import HumanMessage
from langchain_openai import ChatOpenAI
import json

/Users/yaroslav/final-sql/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [4]:
llm_gen_qwen3_coder = ChatOpenAI(
    model="qwen3-coder:30b",  # Указанная модель
    openai_api_base="https://cloud.m1r0.ru/v1",
    openai_api_key="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",
    temperature=0
)
print(f"llm_gen_qwen3_coder инициализирован для модели: {llm_gen_qwen3_coder.model_name}")

llm_gen_qwen3_coder инициализирован для модели: qwen3-coder:30b


In [5]:
with open("trash/questsH.json") as file:
    data = json.load(file)

print(data.keys(), data['queries'][0].keys())

DDL_querys = "\n".join([i['statement'] for i in data['ddl']])
SQL_querys = [i['query'] for i in data['queries']]

defolt_retry_nums = 3
MAX_CONCURRENT_CALLS = 2

dict_keys(['url', 'ddl', 'queries']) dict_keys(['queryid', 'query', 'runquantity', 'executiontime'])


In [6]:
# [CELL 1] ВСТАВИТЬ СРАЗУ ПОСЛЕ ЗАГРУЗКИ ВХОДА В data
# - Определяет текущие <CATALOG>.<SOURCE_SCHEMA> голосованием по полным именам
# - Устанавливает CATALOG, SOURCE_SCHEMA и TARGET_SCHEMA (при необходимости переименуйте)

import re
import json
from collections import Counter
from typing import Dict, Iterable, Tuple

_FQ_TBL = r"[A-Za-z_]\w*\.[A-Za-z_]\w*\.[A-Za-z_]\w*"

def _iter_fq_pairs_from_sql(sql: str) -> Iterable[str]:
    if not sql:
        return ()
    # FROM / JOIN источники
    for pat in (rf"(?i)\bFROM\s+({_FQ_TBL})", rf"(?i)\bJOIN\s+({_FQ_TBL})"):
        for m in re.finditer(pat, sql):
            yield ".".join(m.group(1).split(".")[:2])
    # Явные цели: CREATE/ALTER/INSERT INTO <catalog.schema.table>
    for m in re.finditer(rf"(?i)\b(CREATE|ALTER|INSERT\s+INTO)\s+(?:TABLE\s+)?({_FQ_TBL})", sql):
        yield ".".join(m.group(2).split(".")[:2])

def collect_catalog_schema_pairs(data: Dict) -> Counter:
    pairs: Counter = Counter()
    for d in (data.get("ddl") or []):
        stmt = (d or {}).get("statement") or ""
        for pair in _iter_fq_pairs_from_sql(stmt):
            pairs[pair] += 1
    for q in (data.get("queries") or []):
        sql = (q or {}).get("query") or ""
        for pair in _iter_fq_pairs_from_sql(sql):
            pairs[pair] += 1
    return pairs

def decide_by_vote(pairs: Counter, majority: float = 0.8) -> Tuple[str, str]:
    if not pairs:
        raise ValueError("No fully-qualified table names found to determine <catalog>.<schema>")
    most, cnt = pairs.most_common(1)[0]
    total = sum(pairs.values())
    if cnt == total or (total and cnt / total >= majority):
        cat, sch = most.split(".")
        return cat, sch
    raise ValueError(f"Ambiguous schemas: {dict(pairs)} (no majority >= {majority})")

def detect_catalog_schema(data: Dict, majority: float = 0.8, as_json: bool = False):
    pairs = collect_catalog_schema_pairs(data)
    catalog, schema = decide_by_vote(pairs, majority)
    return {"catalog": catalog, "schema": schema} if as_json else f"{catalog}.{schema}"

# Определяем текущие <catalog>.<schema> и задаём целевую схему
CATALOG, SOURCE_SCHEMA = detect_catalog_schema(data, as_json=False).split('.')
TARGET_SCHEMA = f"{SOURCE_SCHEMA}_v2"  # при необходимости задайте явно, например: 'quests_v2'

print("Detected:", CATALOG, SOURCE_SCHEMA, "-> TARGET:", TARGET_SCHEMA)


Detected: quests public -> TARGET: public_v2


In [7]:
# Утилиты нормализации SQL для сборки результата
import re

def split_sql(sql: str):
    out, current, in_quote = [], [], False
    for ch in sql or "":
        if ch == "'" and (not current or current[-1] != '\\'):
            in_quote = not in_quote
        if ch == ';' and not in_quote:
            stmt = ''.join(current).strip()
            if stmt:
                out.append(stmt)
            current = []
        else:
            current.append(ch)
    tail = ''.join(current).strip()
    if tail:
        out.append(tail)
    return out

def _strip_name(name: str) -> str:
    return name.strip().strip('`').strip('\"')

def ensure_target(name: str) -> str:
    parts = [p for p in _strip_name(name).split('.') if p]
    tbl = parts[-1] if parts else _strip_name(name)
    return f"{CATALOG}.{TARGET_SCHEMA}.{tbl}"

def ensure_source(name: str) -> str:
    parts = [p for p in _strip_name(name).split('.') if p]
    tbl = parts[-1] if parts else _strip_name(name)

    return f"{CATALOG}.{SOURCE_SCHEMA}.{tbl}"

def normalize_ctas_order(stmt: str) -> str:
    pattern = re.compile(r'(?i)(CREATE\s+TABLE\s+[^\s]+)\s+AS\s+(SELECT\b.*)\s+WITH\s*\(([^)]*)\)')

    def repl(match: re.Match) -> str:
        create_part, select_part, with_part = match.group(1), match.group(2), match.group(3)
        return f"{create_part} WITH ({with_part}) AS {select_part}"

    return re.sub(pattern, repl, stmt)

def rewrite_targets_sources(stmt: str) -> str:
    s = stmt
    s = re.sub(r'(?i)\bCREATE\s+TABLE\s+([A-Za-z0-9_\.]+)', lambda m: f"CREATE TABLE {ensure_target(m.group(1))}", s)
    s = re.sub(r'(?i)\bINSERT\s+INTO\s+([A-Za-z0-9_\.]+)', lambda m: f"INSERT INTO {ensure_target(m.group(1))}", s)
    s = re.sub(r'(?i)\bMERGE\s+INTO\s+([A-Za-z0-9_\.]+)', lambda m: f"MERGE INTO {ensure_target(m.group(1))}", s)
    s = re.sub(r'(?i)\bUPDATE\s+([A-Za-z0-9_\.]+)', lambda m: f"UPDATE {ensure_target(m.group(1))}", s)
    s = re.sub(r'(?i)\bFROM\s+([A-Za-z0-9_\.]+)', lambda m: f"FROM {ensure_source(m.group(1))}", s)
    s = re.sub(r'(?i)\bJOIN\s+([A-Za-z0-9_\.]+)', lambda m: f"JOIN {ensure_source(m.group(1))}", s)
    s = re.sub(r'(?i)\bUSING\s+([A-Za-z0-9_\.]+)', lambda m: f"USING {ensure_source(m.group(1))}", s)
    s = normalize_ctas_order(s)
    return ' '.join(s.split())

def qualify_query_to_target(sql: str) -> str:
    s = (sql or '').replace(f"{CATALOG}.{SOURCE_SCHEMA}.", f"{CATALOG}.{TARGET_SCHEMA}.")
    return ' '.join(s.split())


In [8]:
def query_analize_prompt() -> str:
    """
    Упрощённый промпт для query_analize_agent.
    Агент анализирует до 5 SQL-запросов и возвращает текстовое описание закономерностей,
    часто используемых таблиц, соединений, фильтров и потенциальных точек оптимизации под DDL.
    """
    return """Ты — SQL-аналитик для системы оптимизации Data Lakehouse (Trino + Iceberg + S3).
Вход: до 5 SQL-запросов.  
Твоя цель — описать словами структуру и частые паттерны этих запросов, указав:
- какие таблицы чаще всего участвуют;
- какие поля часто используются в фильтрах (WHERE/ON);
- по каким полям часто происходит соединение (JOIN);
- какие операции чаще всего выполняются (GROUP BY, ORDER BY, DISTINCT, WINDOW);
- какие конструкции создают нагрузку или могут быть оптимизированы через DDL;
- какие улучшения можно предложить на уровне DDL (партиционирование, денормализация, сортировка и т. п.).

### Важно:
0. Обращай внимание на частоту выполнения запроса и его время, чтобы в первую очередь оптимизировать тяжёлые запросы
1. **Запрещено** предлагать или использовать materialized views.
2. **Запрещено** менять DDL или писать SQL-код миграций. Только описывать, что требует оптимизации.  
3. **Запрещено** придумывать статистику, размеры таблиц или время выполнения. Если данных нет — укажи "неизвестно".  
4. **Не упоминай** индексы в классическом RDBMS-смысле (B-Tree и т. д.).  
5. **Не упоминай** безопасность, авторизацию, шифрование и внешние системы.  
6. Используй формулировки, применимые к Trino + Iceberg + S3.  
   (Например: «можно рассмотреть партиционирование по дате» вместо «создать индекс».)  
7. Не возвращай JSON, таблицы или списки ключей. Пиши связный текстовый отчёт с пунктами и примерами.

### Структура ответа:
- Краткое резюме: какие таблицы и поля чаще всего встречаются.  
- Анализ соединений: какие таблицы чаще соединяются между собой, по каким ключам.  
- Анализ фильтров: какие колонки часто участвуют в WHERE/ON (например, event_date, user_id).  
- Анализ операций: какие операции создают нагрузку (JOIN, GROUP BY, DISTINCT, ORDER BY, WINDOW).

Вывод должен быть понятным, логически структурированным и готовым для последующей обработки другой моделью, которая будет строить DDL и миграции на основе твоего анализа."""


In [9]:
def query_summarize_prompt() -> str:
    """
    Возвращает промпт (на русском) для query_summarize_agent.
    Агент получает агрегированные результаты от query_analize_agent (строка/батч)
    и должен выдать компактный, читаемый человеком и машиной, сводный отчёт
    — пригодный для подачи в DDL_agent и migrations_agent.
    """

    return """Ты — агрегирующий аналитик SQL для пайплайна оптимизации Data Lakehouse (стек: Trino + Iceberg + S3).

ВХОД:
- Строка / текст `analize` — объединённые результаты работы query_analize_agent для многих батчей (каждый блок содержит анализ до 5 SQL-запросов).
- Формат входа может быть JSON-подобным или текстовым отчётом (см. предыдущий агент). Разбирай оба варианта; если не можешь распарсить фрагмент — помечь как "UNPARSED fragment".

ЦЕЛЬ:
- Объединить и агрегировать мелкие анализы в единый сводный отчёт, выявив:
  1. "Hot" таблицы — таблицы с наибольшей частотой упоминаний в тяжёлых операциях;
  2. Частые пары JOIN (и их join_keys);
  3. Часто используемые фильтры (колонки в WHERE/ON);
  4. Частые тяжёлые операции (GROUP BY, ORDER BY, DISTINCT, WINDOW, SORT);
  5. Повторяющиеся паттерны, которые можно исправить DDL-изменениями;
  6. Очередность (приоритет) изменений для downstream: DDL_agent -> migrations_agent -> query_optimizer.
- Сформировать короткий и однозначный набор рекомендаций уровня DDL (НЕ писать сам DDL/миграции, а только что и где изменить и почему).

СТРОГИЕ ЗАПРЕЩЕНИЯ (выполняй обязательно):
- НЕЛЬЗЯ предлагать или использовать materialized views.
- НЕЛЬЗЯ писать, изменять или генерировать DDL или миграции — это делают downstream агенты.
- НЕЛЬЗЯ придумывать статистику, объёмы данных, кардинальности или время выполнения; если этих метрик нет — указывай `UNKNOWN`.
- НЕЛЬЗЯ предлагать классические RDBMS-индексы (B-Tree и т.п.). Если предлагаешь что-то похожее, опиши его как "файловая/партиц./сортировка/кластеризация в Iceberg" и пометь `depends_on_iceberg_features`.
- НЕЛЬЗЯ обсуждать авторизацию/аутентификацию/безопасность/внешние сервисы.
- Ответ должен быть ТОЛЬКО текстом отчёта (без генерации JSON/DDL/SQL). Пиши структурированный человекочитаемый отчёт (см. формат ниже).

ПРАВИЛА АГРЕГАЦИИ:
- Подсчитывай частоту встречаемости сущностей (таблиц, колонок, join-пар) по данным `analize`. Если входы не содержат явного счётчика — используй относительную частоту (high/medium/low) основанную на количестве вхождений в анализах; если нельзя определить — ставь `UNKNOWN`.
- Если множество анализов указывает на фильтрацию по колонке `event_date`/`ds` — предлагай PARTITIONING (указывай день/месяц как опции). Добавляй замечание о риске при высокой кардинальности.
- Если одна и та же пара таблиц соединяется очень часто — размышляй о DENORMALIZATION (опиши, какие поля включить), но НЕ генерируй CREATE TABLE.
- Для ORDER BY + LIMIT — указывай возможность SORT_ORDER / предварительной кластеризации в Iceberg и помечай `depends_on_iceberg_features`.
- В каждой рекомендации указывай: цель (что менять), конкретные колонки/таблицы, краткая причина (основанную на evidence fragments из `analize`), ориентировочный impact (HIGH/MEDIUM/LOW/UNKNOWN) и какие метрики/проверки нужны (data_volume, cardinality, query_frequency, avg_runtime).
"""


In [10]:
def ddl_optim_prompt() -> str:
    """
    Промпт для агента ddl_optimize_agent.
    Цель — на основе анализа (query_summarize) и оригинальных DDL (ddl_orig)
    выдать только новые или модифицированные DDL-запросы, оптимизирующие работу под Trino + Iceberg + S3.
    Без пояснений, текста и комментариев.
    """
    return f"""Ты — DDL-оптимизатор для Trino + Iceberg + S3.

[Контекст выполнения]
- Catalog: {CATALOG}
- Source schema (исходные таблицы): {SOURCE_SCHEMA}
- Target schema (новая структура): {TARGET_SCHEMA}

[Обязательные правила]
- Полная квалификация всех таблиц: {CATALOG}.<schema>.<table>
- Исходные таблицы в {CATALOG}.{SOURCE_SCHEMA}.* НЕ изменять и не использовать для записи:
  запрещены ALTER/DROP/INSERT/UPDATE/MERGE/DELETE/CREATE VIEW/MATERIALIZED VIEW/GRANT/REVOKE для этой схемы.
- Все новые объекты создавать и изменять ТОЛЬКО в {CATALOG}.{TARGET_SCHEMA}.*.
- Первая DDL-команда должна быть: CREATE SCHEMA {CATALOG}.{TARGET_SCHEMA}.
- Для CTAS соблюдай синтаксис Trino: пишем `CREATE TABLE <...> WITH (...) AS SELECT ...` (WITH перед AS) и не помещаем текст SELECT внутрь partitioning/ARRAY. {CATALOG}.{TARGET_SCHEMA}.
- Не использовать materialized view и несоответствующие Trino/Iceberg свойства.

Вход:
- query_summarize: анализ частых фильтров, сортировок, JOIN
- ddl_orig: исходные DDL таблиц

Задача: Создать оптимизированные DDL для Trino + Iceberg.

СИНТАКСИС Trino + Iceberg:
- Таблицы: <catalog>.<schema>.<table>
- Формат хранения файлов: используйте WITH (format = 'PARQUET')
- Партиционирование (Trino/Iceberg): в CREATE TABLE через WITH (partitioning = ARRAY['day(ts_col)'] или 'month(ts_col)']). Не комбинировать year/month/day на одном столбце.
- Свойства: только допустимые Trino/Iceberg ключи в WITH (...). Не использовать 'write.target-file-size-bytes'. Без висячих запятых.

РАЗРЕШЕНО:
- PARTITIONING (через WITH) с допустимыми функциями
- CREATE TABLE AS SELECT WITH (format = 'PARQUET')
- Создание оптимизированных копий

ЗАПРЕЩЕНО:
- Materialized Views, индексы
- DROP, DELETE, RENAME
- Несовместимый синтаксис

ПРИМЕРЫ:
ALTER TABLE analytics.sales.orders PARTITIONING (через WITH) year(order_date);
CREATE TABLE analytics.sales.orders_new AS SELECT * FROM orders WITH (format = 'PARQUET');
ALTER TABLE analytics.sales.orders -- removed unsafe SET PROPERTIES example

ПРАВИЛА ВЫВОДА:
- Каждый оператор — отдельной строкой; не объединяй множество операторов в одну строку.
- Полные имена таблиц (<catalog>.<schema>.<table>) согласованы с JDBC url.
- Никаких комментариев, префиксов типа "sql ". Только чистые SQL-операторы.

ВЫВОД: Только SQL DDL команды, готовые к выполнению в Trino."""

In [11]:
def migrations_creator_prompt() -> str:
    """
    Промпт для агента migrations_creator_agent.
    Агент получает:
      - query_summarize — сводный анализ SQL-запросов,
      - new_ddl — новые DDL-запросы, сгенерированные ddl_optim_agent.
    Его задача — сгенерировать SQL-миграции для применения новых DDL в безопасном формате,
    совместимом с Trino + Iceberg + S3.
    """
    return f"""Ты — генератор миграций для Trino + Iceberg + S3.

[Контекст выполнения]
- Catalog: {CATALOG}
- Source schema (исходные таблицы): {SOURCE_SCHEMA}
- Target schema (новая структура): {TARGET_SCHEMA}

[Обязательные правила]
- Полная квалификация всех таблиц: {CATALOG}.<schema>.<table>
- Исходные таблицы в {CATALOG}.{SOURCE_SCHEMA}.* НЕ изменять и не использовать для записи:
  запрещены ALTER/DROP/INSERT/UPDATE/MERGE/DELETE/CREATE VIEW/MATERIALIZED VIEW/GRANT/REVOKE для этой схемы.
- Миграции пишут данные ТОЛЬКО в {CATALOG}.{TARGET_SCHEMA}.*, читают ТОЛЬКО из {CATALOG}.{SOURCE_SCHEMA}.*
- Не дублировать DDL из new_ddl: в миграциях не должно быть CREATE TABLE/CTAS уже присутствующих в new_ddl.
- Не использовать materialized view, несоответствующие Trino/Iceberg свойства, авторизацию/GRANT/REVOKE.
- Возвращай только SQL-команды миграций. Без пояснений, без комментариев. Каждая команда на новой строке.

ВХОД:
- query_summarize: анализ использования таблиц
- new_ddl: оптимизированные DDL-запросы

ЦЕЛЬ:
Сгенерировать безопасные SQL-миграции для применения DDL в Trino + Iceberg.

КРИТИЧЕСКИ ВАЖНЫЕ ПРАВИЛА СИНТАКСИСА:
1. ВСЕ таблицы: {CATALOG}.<schema>.<table>
2. ТОЛЬКО совместимый с Trino + Iceberg синтаксис
3. НЕТ materialized views, индексов, деструктивных операций
4. Формат файлов: используем WITH (format = 'PARQUET')

РАЗРЕШЕННЫЕ ОПЕРАЦИИ МИГРАЦИИ:
- INSERT INTO {CATALOG}.{TARGET_SCHEMA}.<table_new>
  SELECT ... FROM {CATALOG}.{SOURCE_SCHEMA}.<table_old>
- UPDATE/MERGE для целевых таблиц в {CATALOG}.{TARGET_SCHEMA}.*
- ALTER TABLE только для {CATALOG}.{TARGET_SCHEMA}.* (например, свойства), если требуется для совместимости
- Валидационные проверки допускаются только как часть миграционного шага (например, INSERT с агрегацией), но не отдельные SELECT

ПОРЯДОК МИГРАЦИЙ:
1. Перенос данных в новые таблицы (INSERT/UPDATE/MERGE) — цели: {CATALOG}.{TARGET_SCHEMA}.*
2. Изменение свойств только новых таблиц (если необходимо)
3. (Опционально) Легкая валидация, но без чистых SELECT

ПРИМЕР ВАЛИДНОГО СИНТАКСИСА:
INSERT INTO {CATALOG}.{TARGET_SCHEMA}.orders
SELECT * FROM {CATALOG}.{SOURCE_SCHEMA}.orders;

MERGE INTO {CATALOG}.{TARGET_SCHEMA}.orders t
USING ({CATALOG}.{SOURCE_SCHEMA}.orders_delta) s
ON t.id = s.id
WHEN MATCHED THEN UPDATE SET amount = s.amount
WHEN NOT MATCHED THEN INSERT (id, amount) VALUES (s.id, s.amount);

ВЫВОД:
ТОЛЬКО SQL-команды миграций, готовые к выполнению в Trino.
Без комментариев, пояснений, текста.
Каждая команда на новой строке."""


In [12]:
def query_optimize_prompt() -> str:
    """
    Промпт для агента query_optimizer.
    Агент получает новые DDL-запросы (new_ddl) и один исходный SQL-запрос (query),
    и должен переписать данный запрос так, чтобы он был оптимизирован под новую структуру данных
    (DDL), сохраняя исходную бизнес-логику и корректность результатов.
    Работает в контексте Trino + Iceberg + S3.
    """
    return f"""Ты — SQL-оптимизатор для Trino + Iceberg + S3.

[Контекст выполнения]
- Catalog: {CATALOG}
- Source schema (исходные таблицы): {SOURCE_SCHEMA}
- Target schema (новая структура): {TARGET_SCHEMA}

[Обязательные правила]
- Полная квалификация всех таблиц: {CATALOG}.<schema>.<table>
- В исходную схему {CATALOG}.{SOURCE_SCHEMA}.* НЕ писать и её не изменять
  (запрещены ALTER/DROP/INSERT/UPDATE/MERGE/DELETE/CREATE VIEW/MATERIALIZED VIEW/GRANT/REVOKE).
- Переписывай запрос так, чтобы он обращался ТОЛЬКО к {CATALOG}.{TARGET_SCHEMA}.* (новая структура).
- Используй новые таблицы/колонки из new_ddl (денормализацию, партиции и т.д.), сохраняя бизнес-логику.
- Синтаксис строго Trino + Iceberg; никаких materialized view, временных таблиц и т.п.

ВХОД:
- new_ddl: новые DDL таблиц (партиции, денормализации, свойства)
- query: исходный SQL запрос для оптимизации

ЦЕЛЬ:
Переписать SQL запрос для использования новой структуры таблиц из DDL.
Сохранить идентичную бизнес-логику и результат.

ПРАВИЛА ОПТИМИЗАЦИИ:
1. Используй новые таблицы и колонки из {CATALOG}.{TARGET_SCHEMA}.*
2. Если в new_ddl есть денормализованные таблицы — убирай лишние JOIN и обращайся к денормализованным колонкам.
3. Для партиционированных таблиц — добавляй/сохраняй фильтры по полям партиций (day()/month() и т.п.), если они присутствуют по смыслу в исходном запросе.
4. Сохраняй все агрегации, фильтры, группировки и логику исходного запроса.
5. Полная квалификация всех таблиц обязательна: {CATALOG}.{TARGET_SCHEMA}.<table> с алиасами.

ЗАПРЕЩЕНО:
- Менять бизнес-логику (WHERE, JOIN, GROUP BY, HAVING) сверх необходимой адаптации к новой структуре.
- Materialized Views, индексы, временные таблицы, любые DDL или операции записи в {CATALOG}.{SOURCE_SCHEMA}.*.
- Комментарии, пояснения, любой текст кроме одного SQL.

АНТИ-ОШИБКИ (строго соблюдать):
- Каждый источник в FROM/JOIN обязан иметь алиас; все колонки должны быть квалифицированы алиасом.
- Нельзя ссылаться на несуществующие поля или алиасы; используй реальные поля исходной/новой таблицы.
- Любой подзапрос/CTE — с алиасом: (...) AS sub.
- GROUP BY: перечисляй только реально присутствующие в SELECT выражения/поля.
- Скобки/запятые: без висячих запятых и лишних скобок.
- Избегай недетерминизма: ORDER BY random() использовать только если требуется задачей.

ВЫВОД: Только один оптимизированный SQL-запрос для Trino (SELECT/CTE), без комментариев.
"""


In [13]:
def critic_prompt() -> str:
    """
    Prompt for LLM‑critic: given original/optimized SQL and other data, return a fixed SQL.
    """
    return f"""Ты — LLM‑критик SQL для Trino + Iceberg.

[Контекст выполнения]
- Catalog: {CATALOG}
- Source schema (исходные таблицы): {SOURCE_SCHEMA}
- Target schema (новая структура): {TARGET_SCHEMA}

[Обязательные правила формата]
- Полная квалификация таблиц: {CATALOG}.<schema>.<table>.
- Исходную схему {CATALOG}.{SOURCE_SCHEMA}.* НЕ изменять и не использовать для записи
  (запрещены ALTER/DROP/INSERT/UPDATE/MERGE/DELETE/CREATE VIEW/MATERIALIZED VIEW/GRANT/REVOKE).
- Оптимизированный запрос должен ссылаться ТОЛЬКО на {CATALOG}.{TARGET_SCHEMA}.*.
  Если optimized использует старую схему — перепиши на {CATALOG}.{TARGET_SCHEMA}.* без изменения бизнес‑логики.
- Никаких DDL/DML в ответе: только один SQL‑запрос (SELECT/CTE) или строка "OK".
- Запрещено: materialized view, любая авторизация/GRANT/REVOKE.

ВХОД (plain‑text блоки, без JSON):

old_ddl:
<CREATE TABLE ...>  (по одному на строку; опционально)

new_ddl:
<CREATE TABLE ...>  (по одному на строку; опционально)

migrations:
<CTAS/INSERT INTO ... SELECT ...>  (перенос из старой структуры в новую; опционально)

original:
<исходный SQL>

optimized:
<оптимизированный SQL>

ЦЕЛЬ:
- Самостоятельно найти и исправить ошибки в поле optimized, если они конечно присутствуют. Проведи собственную многошаговую проверку консистентности, используя при наличии DDL (ddl_old/ddl_new) как источник истины по колонкам и таблицам.
- Исходный запрос (original) практически всегда валиден. Разрешено переиспользовать его конструкции (алиасы, выражения, подзапросы, группировки) во втором запросе, но обязательно с учётом уже выполненной оптимизации (например, денормализация, замена источников, добавление партиционных фильтров).
- Если ошибок НЕТ — верни строку "OK" (без дополнительных слов).
- Если ошибки ЕСТЬ — верни ТОЛЬКО один исправленный SQL‑запрос, без комментариев и текста.

СТРОГИЙ ЧЕК‑ЛИСТ ПРОВЕРКИ:
1) Алиасы: каждый источник в FROM/JOIN имеет алиас; все колонки квалифицированы правильным алиасом.
2) Существование колонок: alias.column существует согласно DDL соответствующей таблицы; при денормализации используй новые таблицы из ddl_new. Если DDL отсутствуют — делай правки только по очевидным несоответствиям (алиасы/скобки/Group By), не придумывай колонки.
3) JOIN‑ключи: ссылки только на колонки, реально присутствующие в соответствующих таблицах; не используй client_id у таблицы, где его нет.
4) Агрегации: GROUP BY соответствует SELECT (все неагрегированные выражения перечислены), нет мусорных токенов (например, 'b1').
5) Синтаксис: нет лишних/незакрытых скобок, висячих запятых. Синтаксис строго Trino.
6) Детерминизм: избегай ORDER BY random() без явной необходимости.
7) Схемы: оптимизированный запрос использует ТОЛЬКО {CATALOG}.{TARGET_SCHEMA}.*; никаких ссылок на {CATALOG}.{SOURCE_SCHEMA}.*.

ВЫВОД:
- Если ошибок нет: ровно "OK".
- Если есть ошибки: только исправленный SQL.
"""


In [14]:
def judge_prompt() -> str:
    """
    Prompt for LLM‑judge: decide if optimized preserves original semantics from structure only.
    """
    return f"""Ты — LLM‑судья эквивалентности SQL (Trino).

[Контекст выполнения]
- Catalog: {CATALOG}
- Source schema (исходные таблицы): {SOURCE_SCHEMA}
- Target schema (новая структура): {TARGET_SCHEMA}

[Обязательные правила формата]
- Полная квалификация таблиц: {CATALOG}.<schema>.<table>.
- Исходную схему {CATALOG}.{SOURCE_SCHEMA}.* НЕ изменять и не использовать для записи
  (запрещены ALTER/DROP/INSERT/UPDATE/MERGE/DELETE/CREATE VIEW/MATERIALIZED VIEW/GRANT/REVOKE).
- Оптимизированный запрос должен ссылаться ТОЛЬКО на {CATALOG}.{TARGET_SCHEMA}.* (новая структура).
- Запрещено: materialized view, любая авторизация/GRANT/REVOKE.
- Если optimized ссылается на старую схему, но семантика идентична — перепиши на {CATALOG}.{TARGET_SCHEMA}.* без изменения бизнес‑логики.

ВХОД (plain‑text блоки, без JSON):

old_ddl:
<CREATE TABLE ...>  (по одному на строку; опционально)

new_ddl:
<CREATE TABLE ...>  (по одному на строку; опционально)

migrations:
<CTAS/INSERT INTO ... SELECT ...>  (перенос из старой в новую; опционально)

original:
<исходный SQL>

optimized:
<оптимизированный SQL>

ЗАДАЧА:
- По структуре SQL (без выполнения) оцени, сохраняет ли optimized бизнес‑логику original (те же агрегаты, фильтры, соединения, проекции) с допустимыми адаптациями под оптимизацию (денормализация, партиционные фильтры и т. п.).
- Учитывай DDL/миграции: если optimized читает данные из новых таблиц (по CTAS/INSERT), интерпретируй соответствия полей между старой и новой структурой.
- При необходимости скорректируй optimized (только SQL) так, чтобы он использовал {CATALOG}.{TARGET_SCHEMA}.* и сохранял семантику original.

СТРОГИЙ ЧЕК‑ЛИСТ:
1) Схемы: optimized использует ТОЛЬКО {CATALOG}.{TARGET_SCHEMA}.*; отсутствие ссылок на {CATALOG}.{SOURCE_SCHEMA}.*.
2) Алиасы: у каждого источника в FROM/JOIN есть алиас; колонки квалифицированы корректным алиасом.
3) Поля: alias.column существуют согласно DDL соответствующих таблиц; при денормализации используй новые таблицы из new_ddl.
4) JOIN‑ключи: соответствуют реальным полям обеих таблиц.
5) Агрегации: GROUP BY согласован с SELECT; без мусорных токенов.
6) Синтаксис: нет лишних/незакрытых скобок, висячих запятых; синтаксис Trino.
7) Детерминизм: избегай ORDER BY random() без явной необходимости.

ВЫВОД:
- Если ошибок нет: ровно "OK".
- Если есть проблема (семантика/структура/синтаксис/схема): верни полный исправленный SQL (один запрос, без комментариев/объяснений).
"""


In [15]:
import asyncio

async def get_answer_async(agent, querys_batch, retry_nums=3):
    for _ in range(retry_nums):
        try:
            prompt = HumanMessage(
                content='\n\n'.join(querys_batch)
            )
            response = await agent.ainvoke({'messages': prompt})
            return response["messages"][-1].content
        except Exception as e:
            print(f"Failed to execute. Error: {repr(e)}")
            print("Retrying...")
    print("OMG_MODEL_CRUSHD!!!")
    return "OMG_MODEL_CRUSHD!!!"

semaphore = asyncio.Semaphore(MAX_CONCURRENT_CALLS)

async def bounded_get_answer_async(agent, batch):
    async with semaphore:
        return await get_answer_async(agent, batch)

query_analize_agent = create_react_agent(
    llm_gen_qwen3_coder,
    tools=[],
    prompt=query_analize_prompt(),
)

querys_batch_prompt = [f"sql-запрос: {i['query']}, количество выполнения:{i['runquantity']}, колличество затраченного времени при едином выполнении запроса{i['executiontime']}" for i in data['queries']]

batches = [
    querys_batch_prompt[5 * i : 5 * (i + 1)]
    for i in range(len(querys_batch_prompt) // 5 + int(len(querys_batch_prompt) % 5 != 0))
]

tasks = [bounded_get_answer_async(query_analize_agent, batch) for batch in batches]

results = await asyncio.gather(*tasks)

analize = "\n\n".join(results)
print(analize)

### Краткое резюме

Наиболее часто используемые таблицы в запросах:  
- `s_client_personal_info` — используется в 4 из 5 запросов, содержит данные о клиентах, включая возраст и источник регистрации.  
- `l_payment_client` — участвует в 4 запросах, связывает платежи с клиентами.  
- `h_client` — участвует в 3 запросах, содержит информацию о клиенте, включая дату создания.  
- `s_client_geo_info` — участвует в 4 запросах, содержит географические данные клиентов.  
- `l_excursion_payment` и `l_quest_payment` — участвуют в 2 запросах, отражают данные о покупках экскурсий и квестов.  
- `s_excursion_geo_info` — участвует в 1 запросе, содержит информацию о географии экскурсий.

Часто используемые поля:  
- `client_id`, `payment_id`, `id` — ключевые поля для соединений.  
- `payment_dt`, `created_at` — даты, часто участвуют в фильтрах и агрегациях.  
- `region`, `city`, `language` — географические поля, часто используются в GROUP BY и фильтрах.  
- `age` — используется для группировки по возр

In [16]:
# summaraize analize from querys

def get_answer(agent, querys_analize):
    for _ in range(defolt_retry_nums):
        try:
            prompt = HumanMessage(
                content= f"Анализ полученных sql запросов{querys_analize}"
            )
            return agent.invoke({'messages': prompt})["messages"][-1].content
        except BaseException as e:
            print(f"Failed to execute. Error: {repr(e)}")
            print("Do retry")
    print("OMG_MODEL_CRUSHD!!!")
    
    return "OMG_MODEL_CRUSHD!!!"

query_summarize_agent = create_react_agent(
    llm_gen_qwen3_coder,
    tools=[],
    prompt=query_summarize_prompt(),
)

query_summarize = get_answer(query_summarize_agent, analize)
print(query_summarize)

### Сводный отчёт по анализу SQL-запросов

#### 1. "Hot" таблицы (наиболее часто упоминаемые в тяжёлых операциях)
- `l_payment_client` — участвует в 5 запросах, используется в JOIN и агрегациях.
- `s_client_personal_info` — участвует в 4 запросах, часто в соединениях и GROUP BY.
- `s_client_geo_info` — участвует в 4 запросах, используется в JOIN и фильтрах.
- `h_client` — участвует в 3 запросах, часто в соединениях.
- `l_excursion_payment` и `l_quest_payment` — участвуют в 2 запросах, используются в JOIN.
- `s_excursion_geo_info` — участвует в 1 запросе, содержит географию экскурсий.

#### 2. Частые пары JOIN и их ключи
- `l_payment_client` ↔ `h_client` по `client_id` — 4 раза.
- `l_payment_client` ↔ `s_client_geo_info` по `client_id` — 4 раза.
- `l_excursion_payment` ↔ `l_payment_client` по `payment_id` — 2 раза.
- `l_quest_payment` ↔ `l_payment_client` по `payment_id` — 2 раза.
- `s_client_personal_info` ↔ `h_client` по `client_id` — 2 раза.
- `s_client_personal_info` ↔ `s_client_geo

In [17]:
# DDL optimize by analize and orig_ddl

def clean_sql(s: str) -> str:
    # Убираем возможные кодовые блоки и обрезаем пробелы
    if s is None:
        return ""
    s2 = s.strip()
    # частые варианты окружения
    s2 = s2.replace("```sql", "```")
    if s2.startswith("```") and s2.endswith("```"):
        s2 = s2[3:-3].strip()
    return s2

def get_answer(agent, query_summarize, ddl_orig):
    for _ in range(defolt_retry_nums):
        try:
            prompt = HumanMessage(
                content=(
                    f"[Context]\n"
                    f"Catalog: {CATALOG}\n"
                    f"Source schema: {SOURCE_SCHEMA}\n"
                    f"Target schema: {TARGET_SCHEMA}\n\n"
                    f"Анализ полученных sql запросов:\n{query_summarize}\n\n"
                    f"Это оригинальные DDL запросы:\n{DDL_querys}"
                )
            )
            resp = agent.invoke({'messages': prompt})["messages"][-1].content
            return clean_sql(resp)
        except BaseException as e:
            print(f"Failed to execute. Error: {repr(e)}")
            print("Do retry")
    print("OMG_MODEL_CRUSHD!!!")
    return "OMG_MODEL_CRUSHD!!!"

ddl_optimize_agent = create_react_agent(
    llm_gen_qwen3_coder,
    tools=[],
    prompt=ddl_optim_prompt(),
)

New_DDL = get_answer(ddl_optimize_agent, query_summarize, DDL_querys)
print(New_DDL)


CREATE SCHEMA quests.public_v2;

CREATE TABLE quests.public_v2.h_author AS SELECT * FROM quests.public.h_author WITH (format = 'PARQUET');

CREATE TABLE quests.public_v2.h_category AS SELECT * FROM quests.public.h_category WITH (format = 'PARQUET');

CREATE TABLE quests.public_v2.h_client AS SELECT * FROM quests.public.h_client WITH (format = 'PARQUET');

CREATE TABLE quests.public_v2.h_episode AS SELECT * FROM quests.public.h_episode WITH (format = 'PARQUET');

CREATE TABLE quests.public_v2.h_episode_completion AS SELECT * FROM quests.public.h_episode_completion WITH (format = 'PARQUET');

CREATE TABLE quests.public_v2.h_excursion AS SELECT * FROM quests.public.h_excursion WITH (format = 'PARQUET');

CREATE TABLE quests.public_v2.h_payment AS SELECT * FROM quests.public.h_payment WITH (format = 'PARQUET');

CREATE TABLE quests.public_v2.h_promo AS SELECT * FROM quests.public.h_promo WITH (format = 'PARQUET');

CREATE TABLE quests.public_v2.h_quest AS SELECT * FROM quests.public.h_ques

In [18]:
# create migrations on analize and new_ddl

def get_answer(agent, query_summarize, new_ddl):
    for _ in range(defolt_retry_nums):
        try:
            prompt = HumanMessage(
                content=(
                    f"[Context]\n"
                    f"Catalog: {CATALOG}\n"
                    f"Source schema: {SOURCE_SCHEMA}\n"
                    f"Target schema: {TARGET_SCHEMA}\n\n"
                    f"Анализ полученных sql запросов:\n{query_summarize}\n\n"
                    f"Это Новые DDL запросы:\n{new_ddl}"
                )
            )
            resp = agent.invoke({'messages': prompt})["messages"][-1].content
            return clean_sql(resp)
        except BaseException as e:
            print(f"Failed to execute. Error: {repr(e)}")
            print("Do retry")
    print("OMG_MODEL_CRUSHD!!!")
    return "OMG_MODEL_CRUSHD!!!"

migrations_creator_agent = create_react_agent(
    llm_gen_qwen3_coder,
    tools=[],
    prompt=migrations_creator_prompt(),
)

migrations = get_answer(migrations_creator_agent, query_summarize, New_DDL)
print(migrations)


INSERT INTO quests.public_v2.h_author SELECT * FROM quests.public.h_author;
INSERT INTO quests.public_v2.h_category SELECT * FROM quests.public.h_category;
INSERT INTO quests.public_v2.h_client SELECT * FROM quests.public.h_client;
INSERT INTO quests.public_v2.h_episode SELECT * FROM quests.public.h_episode;
INSERT INTO quests.public_v2.h_episode_completion SELECT * FROM quests.public.h_episode_completion;
INSERT INTO quests.public_v2.h_excursion SELECT * FROM quests.public.h_excursion;
INSERT INTO quests.public_v2.h_payment SELECT * FROM quests.public.h_payment;
INSERT INTO quests.public_v2.h_promo SELECT * FROM quests.public.h_promo;
INSERT INTO quests.public_v2.h_quest SELECT * FROM quests.public.h_quest;
INSERT INTO quests.public_v2.h_review SELECT * FROM quests.public.h_review;
INSERT INTO quests.public_v2.h_session SELECT * FROM quests.public.h_session;
INSERT INTO quests.public_v2.l_author_quest SELECT * FROM quests.public.l_author_quest;
INSERT INTO quests.public_v2.l_client_ep

In [19]:
codect_info = {
    'old_ddl': DDL_querys,
    'new_ddl': New_DDL,
    'migrations': migrations
}

judge_agent = create_react_agent(
    llm_gen_qwen3_coder,
    tools=[],
    prompt=judge_prompt(),
)
critic_agent = create_react_agent(
    llm_gen_qwen3_coder,
    tools=[],
    prompt=critic_prompt(),
)

global codect_info, judge_agent, critic_agent

async def cheak_query(old_querys, new_querys):
    global codect_info, judge_agent, critic_agent

    # JUDGE
    prompt = HumanMessage(
        content=(
            f"[Context]\n"
            f"Catalog: {CATALOG}\n"
            f"Source schema: {SOURCE_SCHEMA}\n"
            f"Target schema: {TARGET_SCHEMA}\n\n"
            f"старые DDL запросы:\n{codect_info['old_ddl']}\n\n"
            f"Новые DDL запросы:\n{codect_info['new_ddl']}\n\n"
            f"Вот миграции:\n{codect_info['migrations']}\n\n"
            f"оригинальный sql запрос:\n{old_querys}\n\n"
            f"оптимизированный запрос:\n{new_querys}\n"
        )
    )
    judge_response = await judge_agent.ainvoke({'messages': prompt})
    cheak_logik = clean_sql(judge_response["messages"][-1].content)

    if 'OK' not in cheak_logik:
        new_querys = cheak_logik

    # CRITIC
    prompt = HumanMessage(
        content=(
            f"[Context]\n"
            f"Catalog: {CATALOG}\n"
            f"Source schema: {SOURCE_SCHEMA}\n"
            f"Target schema: {TARGET_SCHEMA}\n\n"
            f"старые DDL запросы:\n{codect_info['old_ddl']}\n\n"
            f"Новые DDL запросы:\n{codect_info['new_ddl']}\n\n"
            f"Вот миграции:\n{codect_info['migrations']}\n\n"
            f"оригинальный sql запрос:\n{old_querys}\n\n"
            f"оптимизированный запрос:\n{new_querys}\n"
        )
    )
    critic_response = await critic_agent.ainvoke({'messages': prompt})
    cheak_sintax = clean_sql(critic_response["messages"][-1].content)

    if 'OK' not in cheak_sintax:
        new_querys = cheak_sintax

    return new_querys


In [20]:
# Сборка финального ответа в формате сервиса

# Создаём DDL: первая команда — создание схемы, далее только новые таблицы/изменения
raw_ddl = [stmt for stmt in split_sql(New_DDL) if stmt.strip()]
ddl_entries = []
for stmt in raw_ddl:
    normalized = rewrite_targets_sources(stmt)
    if normalized and not normalized.strip().upper().startswith('SELECT'):
        ddl_entries.append({'statement': normalized})

# Готовим миграции: только DML для переноса данных в новую схему
raw_migrations = [stmt for stmt in split_sql(migrations) if stmt.strip()]
migration_entries = []
for stmt in raw_migrations:
    normalized = rewrite_targets_sources(stmt)
    upper = normalized.strip().upper()
    if upper.startswith(('INSERT', 'UPDATE', 'MERGE')):
        migration_entries.append({'statement': normalized})
optim_querys = globals().get('optim_querys', {})
# Оптимизированные запросы: обращаются только к новой схеме
query_entries = []
for item in data['queries']:
    qid = item['queryid']
    optimized_sql = clean_sql(optim_querys.get(qid, item['query']))
    query_entries.append({'queryid': qid, 'query': qualify_query_to_target(optimized_sql)})

json_answer = {
    'ddl': ddl_entries,
    'migrations': migration_entries,
    'queries': query_entries,
}


In [21]:
with open('final.json', 'w') as fp:
    json.dump(json_answer, fp, indent=4)